# Demo: Synthetic Data Generation and Streamlit Demo

Bu notebook, veri üretimini ve basit bir Streamlit demo uygulamasını örnekler. Aşağıdaki bölümler kod odaklıdır: gereksinimler, rastgelelik kontrolü, temel dağılımlar, zaman serisi, görselleştirme ve küçük bir Prophet tahmin örneği.

Not: Streamlit uygulamasını çalıştırmak için terminalde `streamlit run streamlit_app.py` komutunu kullanın.

## 1) Gereksinimler ve ortam kurulumu
Aşağıdaki paketleri kurun (örn. virtualenv içinde):

```bash
pip install -r requirements.txt
# veya örnek olarak
pip install numpy pandas matplotlib seaborn scikit-learn streamlit prophet plotly pytest
```

In [ ]:
# 2) Rastgelelik ve tekrar üretilebilirlik (seed)
import numpy as np
import random

def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)

set_seed(42)
print('Seeds set')

In [ ]:
# 3) Temel dağılımlardan veri üretimi (Uniform, Normal, Poisson)
import numpy as np
import matplotlib.pyplot as plt

x_norm = np.random.normal(loc=0, scale=1, size=1000)
x_uniform = np.random.uniform(low=-1, high=1, size=1000)
x_poisson = np.random.poisson(lam=3, size=1000)

fig, axs = plt.subplots(1,3, figsize=(12,3))
axs[0].hist(x_norm, bins=30); axs[0].set_title('Normal')
axs[1].hist(x_uniform, bins=30); axs[1].set_title('Uniform')
axs[2].hist(x_poisson, bins=30); axs[2].set_title('Poisson')
plt.tight_layout()
plt.show()

In [ ]:
# 4) Çok değişkenli veri ve korelasyon (multivariate normal)
import pandas as pd
from pandas.plotting import scatter_matrix

cov = [[1.0, 0.8, 0.3],[0.8, 1.0, 0.2],[0.3, 0.2, 1.0]]
X = np.random.multivariate_normal(mean=[0,0,0], cov=cov, size=300)
df_mv = pd.DataFrame(X, columns=['x1','x2','x3'])
scatter_matrix(df_mv, alpha=0.6, figsize=(6,6));
plt.show()

In [ ]:
# 5) Zaman serisi üretimi (trend, sezonsallık, AR(1))
import pandas as pd

def generate_ar1_series(phi=0.7, n=200, sigma=1.0, seed=0):
    np.random.seed(seed)
    eps = np.random.normal(0, sigma, size=n)
    x = np.zeros(n)
    for t in range(1, n):
        x[t] = phi * x[t-1] + eps[t]
    return x

n = 365
trend = np.linspace(0, 10, n)
season = 5 * np.sin(2 * np.pi * np.arange(n) / 365)
ar = generate_ar1_series(phi=0.6, n=n, sigma=1.0, seed=1)
series = trend + season + ar

dates = pd.date_range('2023-01-01', periods=n)
df_ts = pd.DataFrame({'Date': dates, 'Value': series})

plt.figure(figsize=(10,3)); plt.plot(df_ts['Date'], df_ts['Value']); plt.title('Trend + Season + AR(1)'); plt.show()

In [ ]:
# 6) Sınıflandırma ve regresyon için sentetik veri (sklearn)
from sklearn.datasets import make_classification, make_regression

Xc, yc = make_classification(n_samples=300, n_features=2, n_informative=2, n_redundant=0, random_state=0)
Xr, yr = make_regression(n_samples=300, n_features=1, noise=10.0, random_state=0)

plt.figure(figsize=(6,3)); plt.scatter(Xc[:,0], Xc[:,1], c=yc); plt.title('Classification sample'); plt.show()
plt.figure(figsize=(6,3)); plt.scatter(Xr[:,0], yr); plt.title('Regression sample'); plt.show()

In [ ]:
# 7) Gürültü, outlier ve eksik değer ekleme

def add_noise(arr, scale=1.0, seed=None):
    rng = np.random.default_rng(seed)
    return arr + rng.normal(0, scale, size=arr.shape)

arr = np.linspace(0, 10, 100)
arr_noisy = add_noise(arr, scale=0.5, seed=0)

# Add outliers
arr_noisy[::20] += 10

# Add missing
arr_noisy[5] = np.nan

plt.figure(figsize=(6,3)); plt.plot(arr_noisy); plt.title('Noisy series with outliers and missing'); plt.show()

In [ ]:
# 8) Veri kaydetme ve paylaşma (CSV, Parquet, np.save)
# Örnek: df_ts to CSV / Parquet
csv_path = 'data/processed/demo_time_series.csv'
parquet_path = 'data/processed/demo_time_series.parquet'

import os
os.makedirs(os.path.dirname(csv_path), exist_ok=True)

df_ts.to_csv(csv_path, index=False)
df_ts.to_parquet(parquet_path)
print('Saved CSV and Parquet')

In [ ]:
# 9) Görselleştirme (histogram, scatter, pairplot, zaman serisi)
import plotly.express as px

# Daily sales generator from project
from data_generator import generate_fashion_data

df = generate_fashion_data(output_path=None, periods=90, seed=0)

# aggregate by date and product
agg = df.groupby(['Date','Product'])['Sales'].sum().reset_index()
fig = px.line(agg, x='Date', y='Sales', color='Product', title='Daily Sales by Product')
fig.show()

# 10) Streamlit demo uygulaması (parametre sliderları, görselleştirme, indirme)
# Repo içinde `streamlit_app.py` dosyası bulunmaktadır. Streamlit uygulamasını terminalde aşağıdaki komut ile çalıştırabilirsiniz:
# streamlit run streamlit_app.py

print('Streamlit demo hazır: `streamlit_app.py` dosyasını kullanın')

## 11) Birim testler ve VSCode içinde çalışma (pytest)
Test örnekleri projenin `tests/` klasöründe yer alır. `pytest` ile testleri çalıştırabilirsiniz.